In [6]:
import numpy as np
import spiceypy as spice
from lamberthub import izzo2015
from scipy.optimize import differential_evolution

# --------------------------------------------------
# SPICE setup
# --------------------------------------------------
spice.furnsh("/Users/rebnoob/Documents/ae105/generic_kernels/lsk/naif0012.tls")
spice.furnsh("/Users/rebnoob/Documents/ae105/generic_kernels/spk/planets/de442.bsp")

# --------------------------------------------------
# constants
# --------------------------------------------------
MU_SUN   = 132712440041.279  # km^3/s^2
MU_EARTH = 398600.435436
MU_MARS  = 42828.375214
R_EARTH  = 6378.1363
R_MARS   = 3396.19
r_park_earth = R_EARTH + 200.0
r_park_mars  = R_MARS  + 200.0

# fixed arrival date
t_arr = spice.str2et("2036 APR 05")

# --------------------------------------------------
# helper: dv formulas
# --------------------------------------------------
def dv_escape_from_circ(v_inf, mu_body, r_circ):
    v_circ = np.sqrt(mu_body / r_circ)
    v_esc  = np.sqrt(2.0 * mu_body / r_circ)
    v_peri = np.sqrt(v_inf**2 + v_esc**2)
    return v_peri - v_circ

def dv_capture_to_circ(v_inf, mu_body, r_circ):
    v_circ = np.sqrt(mu_body / r_circ)
    v_peri = np.sqrt(v_inf**2 + 2.0 * mu_body / r_circ)
    return v_peri - v_circ

def sun_state(body, et):
    s, _ = spice.spkezr(body, et, "ECLIPJ2000", "NONE", "SUN")
    return np.array(s[:3]), np.array(s[3:6])

# --------------------------------------------------
# universal variable kepler propagator (2-body)
# this is standard f-g propagation
# --------------------------------------------------
def kepler_propagate(r0, v0, dt, mu):
    r0 = np.array(r0, dtype=float)
    v0 = np.array(v0, dtype=float)
    r0_norm = np.linalg.norm(r0)
    v0_norm = np.linalg.norm(v0)
    vr0 = np.dot(r0, v0) / r0_norm

    alpha = 2.0 / r0_norm - v0_norm**2 / mu  # reciprocal of a

    # initial guess for universal anomaly
    if alpha > 0:
        chi = np.sqrt(mu) * dt * alpha
    else:
        chi = np.sign(dt) * np.sqrt(-1.0/alpha) * np.log(-2.0*mu*alpha*dt/(vr0 + np.sign(dt)*np.sqrt(-mu/alpha) * (1 - r0_norm*alpha)))

    def stumpC(z):
        if z > 0:
            return (1 - np.cos(np.sqrt(z))) / z
        elif z < 0:
            return (np.cosh(np.sqrt(-z)) - 1) / -z
        else:
            return 0.5

    def stumpS(z):
        if z > 0:
            return (np.sqrt(z) - np.sin(np.sqrt(z))) / (z ** 1.5)
        elif z < 0:
            return (np.sinh(np.sqrt(-z)) - np.sqrt(-z)) / ((-z) ** 1.5)
        else:
            return 1.0/6.0

    # solve for chi by Newton
    for _ in range(60):
        z = alpha * chi**2
        C = stumpC(z)
        S = stumpS(z)
        r = chi**2 * C + vr0 / np.sqrt(mu) * chi * (1 - z*S) + r0_norm * (1 - z*C)
        f = r - mu**0.5 * dt
        if abs(f) < 1e-8:
            break
        df = chi * (1 - z*S) * (1/np.sqrt(mu)) * (vr0) + (1 - z*C) * (r0_norm) / np.sqrt(mu) + chi**3 * S / np.sqrt(mu)
        # That df expression is messy; simpler is Vallado’s expression:
        df = (chi**2 * C + vr0/np.sqrt(mu)*chi*(1 - z*S) + r0_norm*(1 - z*C)) / (np.sqrt(mu))  # dt/dchi
        chi = chi - (f / df)

    z = alpha * chi**2
    C = stumpC(z)
    S = stumpS(z)
    f = 1 - (chi**2 / r0_norm) * C
    g = dt - 1/np.sqrt(mu) * chi**3 * S
    r = f * r0 + g * v0
    r_norm = np.linalg.norm(r)
    gdot = 1 - (chi**2 / r_norm) * C
    fdot = (np.sqrt(mu) / (r_norm * r0_norm)) * (z*S - 1) * chi
    v = fdot * r0 + gdot * v0

    return r, v

# --------------------------------------------------
# merit function F(x) = DV0 + DVDSM + DV1
# x = [t0, vix, viy, tDSM]
# --------------------------------------------------
def merit(x):
    t0, vix, viy, t_dsm = x

    # enforce DSM between t0 and t_arr
    if t_dsm <= t0 or t_dsm >= t_arr:
        return 1e6  # big penalty

    # 1) departure state
    rE, vE = sun_state("EARTH Barycenter", t0)
    v_inf_vec = np.array([vix, viy, 0.0])
    v0 = vE + v_inf_vec
    r0 = rE

    # 2) propagate to DSM
    dt_dsm = t_dsm - t0
    r_dsm, v_coast = kepler_propagate(r0, v0, dt_dsm, MU_SUN)

    # 3) Lambert from DSM to Mars at fixed arrival
    rM, vM = sun_state("MARS Barycenter", t_arr)
    dt_leg = t_arr - t_dsm
    if dt_leg <= 0:
        return 1e6

    try:
        v_dsm_req, v_arr_req = izzo2015(MU_SUN, r_dsm, rM, dt_leg, M=0, prograde=True)
    except Exception:
        return 1e6

    # 4) DV at DSM
    dv_dsm = np.linalg.norm(v_dsm_req - v_coast)

    # 5) DV0: from 200 km LEO to |v_inf_vec|
    v_inf_mag = np.linalg.norm(v_inf_vec)
    dv0 = dv_escape_from_circ(v_inf_mag, MU_EARTH, r_park_earth)

    # 6) DV1: Mars arrival
    v_inf_arr_vec = v_arr_req - vM
    v_inf_arr = np.linalg.norm(v_inf_arr_vec)
    dv1 = dv_capture_to_circ(v_inf_arr, MU_MARS, r_park_mars)

    return dv0 + dv_dsm + dv1

# --------------------------------------------------
# define bounds and run optimization
# --------------------------------------------------
# choose a reasonable departure window: say 2035-11-01 to 2036-01-15
t0_min = spice.str2et("2035 NOV 01")
t0_max = spice.str2et("2036 JAN 15")

# v-infinity components: let’s allow +/- 4 km/s in-plane
vi_max = 4.0
vi_min = -4.0

# DSM time: must be between t0 and t_arr, but DE needs fixed bounds.
# we can just allow DSM from t0_min+20d up to t_arr-20d
t_dsm_min = t0_min + 20*86400.0
t_dsm_max = t_arr - 20*86400.0

bounds = [
    (t0_min, t0_max),    # t0
    (vi_min, vi_max),    # v_inf_x
    (vi_min, vi_max),    # v_inf_y
    (t_dsm_min, t_dsm_max)  # t_dsm
]

result = differential_evolution(
    merit,
    bounds,
    tol=1e-3,
    maxiter=120,
    polish=True,
)

print("Best F (km/s):", result.fun)
best_t0, best_vix, best_viy, best_t_dsm = result.x
print("Departure:", spice.et2utc(best_t0, "C", 0))
print("DSM time :", spice.et2utc(best_t_dsm, "C", 0))
print("v_inf0   :", best_vix, best_viy, 0.0)

Best F (km/s): 1000000.0
Departure: 2035 DEC 24 20:02:49
DSM time : 2035 DEC 09 13:42:08
v_inf0   : -3.2758685985271274 -3.5954267484000715 0.0
